# Project: Exploring Search Trees and Alpha-Beta Pruning with Chess

## Machine Learning, Fall 2021

### Name: Davin Jeong

### Sources
* [Alpha-beta Pruning in Chess Engines](https://umm-csci.github.io/senior-seminar/seminars/spring2017/marckel.pdf), by Otto Marckel
* [The Anatomy of a Chess AI](https://medium.com/@SereneBiologist/the-anatomy-of-a-chess-ai-2087d0d565), by Levi Walker

### Objective: 
**A Chess AI that can learn from a database of positions in order to suggest strong moves in new positions**

In [18]:
import numpy as np
import chess # Chess library for move generation and move validation: https://python-chess.readthedocs.io/en/latest/

## General Approach
Chess is estimated to have about $10^{120}$ games. To put things into perspective, there are about $10^{80}$ atoms in the observable universe. Its complexity and popularity as a sport has made it a popular subject of computer science research. Some approaches are listed below: 
- An Explicit Algorithm: It might be possible, in theory, to mimic the way humans play by comnining analysis, strategy, and tactics to determine the next move. However, in practice, we cannot program the complex computations humans perform intuitively. 
- Brute Force: The fastest supercomputer performs ["quadrillions of floating point operations per second."](https://asia.nikkei.com/Business/Technology/Japan-s-Fugaku-keeps-position-as-world-s-fastest-supercomputer) For reference, one quadrillion is $10^{15}$. Clearly, it is completely impractical, not to mention unethical, to try to brute force chess. 
- Search Tree: A third approach is best exemplified by the commonly-heard phrase "looking ahead *x* moves." Just as human players create plans by anticipating future moves, evaluating their outcomes, and following the path with most potential, search trees look a limited number of moves ahead and calculate the move that is most likely to result in a win. 

The Alpha-Beta Search Algorithm, proposed by around 1958, was one of the first autonomous chess-playing algorithms. At its core, it relies on a combination of an evaluator function and a search tree. We will focus on this algorithm in this notebook. 

## Evaluating Positions
At their base, search tree algorithms rely on an evaluator function: some way of quantifying the "goodness" of a position. This is crucial since we need some way of looking at a chain of moves and identifying how "good," or likely to lead to a win, for a player.

Evaluator functions can employ any number of approaches. The general idea is to map the influence of certain features on a position's "goodness." These features could be determined however we please, whether it be through heuristics/analysis or using unsupervised, supervised, or reinforcment learning. 



In [17]:
# Evaluate based on material

piece_points = {'R' : 5, 'r' : 5, 
                'N' : 3, 'n' : 3 , 
                'B' : 3, 'b' : 3,
                'Q' : 9, 'q' : 9, 
                'P' : 1, 'p' : 1,
                'K' : 0, 'k' : 0,
                '.' : 0, } 
def evaluate(board):
    whiteMaterial = blackMaterial = 0 
    
    rows = str(board).split('\n')
    for row in rows:
        for piece in row.split(' '):
            #print(piece)
            if piece.isalpha() and piece.isupper():
                whiteMaterial += piece_points[piece]
            else:
                blackMaterial += piece_points[piece]  
    return whiteMaterial - blackMaterial

In [11]:
# Test Above Code

board = chess.Board("r1bqkb1r/pppp1Qpp/2n2n2/4p3/2B1P3/8/PPPP1PPP/RNB1K1NR b KQkq - 0 4") # https://tynedalechess.wordpress.com/2017/11/05/fen-strings-explained/
print(board)

print(f"Material Evaluation: {evaluate(board)}")

r . b q k b . r
p p p p . Q p p
. . n . . n . .
. . . . p . . .
. . B . P . . .
. . . . . . . .
P P P P . P P P
R N B . K . N R
Material Evaluation: 1


**TODO: Create an evaluation function using supervised learning on [Chess Evaluations](https://www.kaggle.com/ronakbadhe/chess-evaluations/code)**

## Search Trees

Search trees are used to model the outcomes of possible moves:

![title](https://www.sites.google.com/site/qgchess/_/rsrc/1467139373444/chess-algorithms/minmaxtree.JPG)
**Image from [QGChess](https://www.sites.google.com/site/qgchess/chess-algorithms)**

In the above diagram, we see a search tree rooted at the starting position of chess (A). Each of it's children nodes (B, C, D) represent potential moves that white can play. The children nodes of those children (E, F) represent black's potential responses. We can use this structure to look a limited number of moves into the future. 

## Minimax Search

Now that we have a way of evaluating a position, our material evaluation, and modelling the possible variations stemming from it, the search tree, we need to combine these to search the tree and make decisions. One such algorithm is Minimax. 

For Minimax, we label one player "the maximizer" and the other "the minimizer." These names stem from the fact that our evaluation tells us how much higher white's score is over black's: white would want to maximize this score, and black would want to minimize it. 

Minimax assumes both players optimally and tries to optimise the worst possible case that could result from the current player's move. 

![title](https://www.baeldung.com/wp-content/uploads/2017/07/minimax.png)
**Image from [Introduction to Minimax](https://www.baeldung.com/java-minimax-algorithm)**

For instance, consider the above image. The search tree is built out from our root node, with each following node representing possible moves of their parent. The leaf nodes represent all the possible positions after a certain amount of moves. For our Chess Bot, we would store the positions' details, and we would have more children per node (about 35). Here, we are trying to illustrate Minimax, so we only see positions' evaluations. 

Here's a breakdown of the algorithm:
1. Build out the search tree, evaluate leaf nodes
2. Build up from there recursively. Each level is either white's or black's turn. White tries to maximize the score by picking the highest evaluation child. Black tries to minimize the score by picking the lowest evaluation child.
3. In the end, we realize that white should pick the move returning the 4 evaluation (the leaf node 2nd from the right), as this is the maximium position it can guarantee. 

## Alpha-Beta Pruning

As you might have guessed, exhausting every position using a search tree, even to a limited depth, is a time-consuming process in a game as complex as chess is. As a result, people have developed many optimizations over the years to reduce the positions we have to evaluate. One of them is Alpha-Beta Pruning. 

This algorithm builds upon Minimax by elaborating on the premise that each player will play optimally. We explore the tree in a depth-first search manner. After following a sequence of moves down to the leaf node, we employ the same bottom up approach as Minimax, with the added component of variables *Alpha* and *Beta*. Alpha represents the best guaranteed option for the maximizing player in the current branch, and beta represents the best guaranteed option for the minimizing player in the current branch. We use these to prune branches that won't be explored, as they're necessarily suboptimal.

![title](https://d1m75rqqgidzqn.cloudfront.net/wp-data/2020/05/08235613/Blog-8-5-2020-04-1024x598.jpg)
**Image from [Alpha Beta Pruning in AI](https://www.mygreatlearning.com/blog/alpha-beta-pruning-in-ai/)**

Here's a breakdown of the algorithm:
1. Depth-first search to a leaf node. Let's say we take A -> B -> D.
2. At each node, store the alpha and beta values. At D, the maximizer can guarantee an evaluation of so alpha is 3, but the minimizer cannot guarantee anything (it has no input), so the beta is infinity.
3. After we calculate this for one node, we travel back up the tree and update our beta and alpha values. So, based on our current knowledge, the maximizer can guarantee an evaluation of 3 at B, and the minimizer can guarantee an evaluation of 3 if it chooses the previously explored path.
4. In-line with normal dfs, we explore another path. We go down from node B to E, and check the first position. Already, the maximizer can force a position of 5. This is already higher than 3 from the previous branch, so the minimizer must force the D branch. We need not further evaluate from the E node. 
5. Repeat this procedure for all other nodes. Eventually, we'll have the best guaranteed outcome for white in the root node, but we will have obtained it by evaluating less positions (using the Alpha-Beta pruning).

In [ ]:
'''
Pseudocode: 

def dfs(board, height, alpha, beta, alphaTurn):
   if height==0 or board.isendofgame():
      return eval(board)
   else if(alphaTurn):
      maxEval = -infinity
      for each continuation:
         eval = dfs(board, height - 1, alpha, beta, False)
         maxEval = max(maxEval, eval)
         alpha = max(alpha, eval)
         # Alpha at this node will be >= its current value. If this is higher than a guaranteed beta, black will go for the other option in the previous turn.
         if alpha >= beta: 
            break
      return maxEval
   else:
      minEval = +infinity
      for each continuation:
         eval = dfs(board, height - 1, alpha, beta, True)
         minEval = min(minEval, eval)
         beta = min(beta, eval)
         # Beta at this node will be <= its current value. If this is lower than a guaranteed alpha, white will go for the other option in the previous turn.
        if beta <= alpha:
           break
       return minEval
'''